In [ ]:
# !pip uninstall kiwisolver matplotlib
# !pip install kiwisolver matplotlib
# !pip install tensorflow[and-cuda]==2.10
# !pip list | grep cud
# import tensorflow as tf
# print(tf.config.list_physical_devices('GPU'))
# !pip install pydub

In [ ]:
### Libraries ###
import os, sys, re, time, json, traceback, logging, datetime, gc, shutil, math, base64, pickle
from datetime import datetime
from pathlib import Path
import numpy as np
import torch
# from numba import cuda
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

### Audio Libraries ###
from pydub import AudioSegment, effects
from pyannote.audio import Pipeline
import torchaudio
from speechbrain.pretrained import EncoderClassifier, SpeakerRecognition
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans, DBSCAN
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from kneed import KneeLocator

In [ ]:
print(torch.cuda.current_device())
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.version())
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
### Path of Files ###
input_video_filename_wo_ext = '1_min_video'
input_video_filename = 'Videos_Examples/'+ input_video_filename_wo_ext +'.mp4' # The video files should be on mp4 format
output_video_path = 'Videos_Output/'

In [ ]:
# Create a new directory of output of videos
if not os.path.exists(output_video_path):
    os.makedirs(output_video_path)

# Create a new directory for processed audio if it does not exist
dir_output_audio = output_video_path + input_video_filename_wo_ext +'/Audio/'
if not os.path.exists(dir_output_audio):
    os.makedirs(dir_output_audio)
dir_output_audio_step1 = output_video_path + input_video_filename_wo_ext +'/Audio/Step1_ProcessAudio/'
if not os.path.exists(dir_output_audio_step1):
    os.makedirs(dir_output_audio_step1)
dir_output_audio_step2 = output_video_path + input_video_filename_wo_ext +'/Audio/Step2_Spleeter/'
if not os.path.exists(dir_output_audio_step2):
    os.makedirs(dir_output_audio_step2)
dir_output_audio_step3 = output_video_path + input_video_filename_wo_ext +'/Audio/Step3_Segments/'
if not os.path.exists(dir_output_audio_step3):
    os.makedirs(dir_output_audio_step3)
dir_output_audio_step4 = output_video_path + input_video_filename_wo_ext +'/Audio/Step4_Embeddings/'
if not os.path.exists(dir_output_audio_step4):
    os.makedirs(dir_output_audio_step4)
dir_output_audio_step5 = output_video_path + input_video_filename_wo_ext +'/Audio/Step5_Clusters/'
if not os.path.exists(dir_output_audio_step5):
    os.makedirs(dir_output_audio_step5)
    
# Create a new directory for processed images if it does not exist
dir_output_images = output_video_path + input_video_filename_wo_ext +'/Images/'
if not os.path.exists(dir_output_images):
    os.makedirs(dir_output_images)
dir_output_images_step1 = output_video_path + input_video_filename_wo_ext +'/Images/Step1_Images/'
if not os.path.exists(dir_output_images_step1):
    os.makedirs(dir_output_images_step1)
dir_output_images_step2 = output_video_path + input_video_filename_wo_ext +'/Images/Step2_Faces/'
if not os.path.exists(dir_output_images_step2):
    os.makedirs(dir_output_images_step2)
dir_output_images_step3 = output_video_path + input_video_filename_wo_ext +'/Images/Step3_Embeddings/'
if not os.path.exists(dir_output_images_step3):
    os.makedirs(dir_output_images_step3)
dir_output_images_step4 = output_video_path + input_video_filename_wo_ext +'/Images/Step4_Clusters/'
if not os.path.exists(dir_output_images_step4):
    os.makedirs(dir_output_images_step4)

# Create a new directory for processed text if it does not exist
dir_output_text = output_video_path + input_video_filename_wo_ext +'/Text/'
if not os.path.exists(dir_output_text):
    os.makedirs(dir_output_text)
dir_output_text_seg = output_video_path + input_video_filename_wo_ext +'/Text/Segments/'
if not os.path.exists(dir_output_text_seg):
    os.makedirs(dir_output_text_seg)

In [ ]:
#####################################################
# Video to Audio
#####################################################

In [ ]:
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1", use_auth_token="")
# pipeline = pipeline.to(0)

In [ ]:
%%timeit
### Segments Separation ###
# Apply pretrained pipeline
diarization = pipeline(dir_output_audio_step2 + 'audio.flac')
timeline = list()
speakers_dict = dict()
for turn, _, speaker in diarization.itertracks(yield_label=True):
    timeline.append([speaker.replace('SPEAKER_', 'SP'), round(turn.start,1), round(turn.end,1)])
    speakers_dict[speaker] = 1

# Export speech part
audio_file = AudioSegment.from_file(file=dir_output_audio_step2 + 'audio.flac', format="flac")

n_clusters = len(speakers_dict)

count = 0
for seg in timeline:
    output_audio = audio_file[seg[1]*1000:seg[2]*1000]
    secs = seg[2]-seg[1]
    if secs > 1:
        output_audio.export(str(dir_output_audio_step3 + 'seg_' + str(count) +'_'+ seg[0] +'_' + str(seg[1]) + '-' + str(seg[2]) +'.flac'), format="flac")
        count += 1

# pipeline = None
torch.cuda.empty_cache()
gc.collect()
# device = cuda.get_current_device()
# device.reset()
print('Finish')

In [ ]:
# from pyannote.audio import Inference, Model, Audio
# from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
# from speechbrain.pretrained import EncoderClassifier, SpeakerRecognition
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")
# model = PretrainedSpeakerEmbedding("speechbrain/spkrec-ecapa-voxceleb", device=torch.device("cuda"))
# audio = Audio(sample_rate=16000, mono=True)

In [ ]:
# %%timeit
### Create Voide Embeddings ###
# if not os.path.exists(dir_output_audio_step4 + 'embeddings.pkl'): # Create embeddings
embeddings = []
embeddings_filename = []

for filename in sorted(os.listdir(dir_output_audio_step3)):
    if '.flac' in filename:
        #embeddings.append(DeepFace.represent(dir_output_audio_step3+filename, enforce_detection=False)[0]['embedding'])
        signal, fs = torchaudio.load(dir_output_audio_step3+filename)
        embeddings.append(classifier.encode_batch(signal, normalize=True)[0][0])
        embeddings_filename.append(filename)

# Store pickle/embeddings
embeddings_all = {'embeddings': embeddings, 'filename': embeddings_filename}

f_pickle = open(dir_output_audio_step4 + 'embeddings.pkl', "wb")
pickle.dump(embeddings_all, f_pickle)
f_pickle.close()

print('Finish.')

In [ ]:
f_pickle = open(dir_output_audio_step4 + 'embeddings.pkl', 'rb')
embeddings_all = pickle.load(f_pickle)

In [ ]:
# from sklearn.metrics import silhouette_score

# clf = DBSCAN(n_jobs=-1, min_samples=1, eps=42, metric='euclidean') # cosine (0.000199) euclidean
# clf.fit(embeddings_all['embeddings'])
# preds = clf.labels_
# print(preds)
# print('Number of clusters:', len(set(preds)))

# # Calculate silhouette score to evaluate the quality of clustering
# silhouette_avg = silhouette_score(embeddings_all['embeddings'], preds)
# print("The average silhouette_score is :", silhouette_avg)

In [ ]:
# %%timeit
### Cluster Voice Embeddings ###
# Delete all clusters folders
for filename in sorted(os.listdir(dir_output_audio_step5)):
    if 'cluster' in filename:
        shutil.rmtree(dir_output_audio_step5+filename)

# Dimensionality reduction for visualization and potentially better clustering
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings_all['embeddings'])
        
# clf = KMeans(n_clusters=n_clusters, n_init=100, max_iter=600, random_state=0)
# if kmeans use number of speakers from voice
clf = DBSCAN(n_jobs=-1, min_samples=2, eps=5.5, metric='euclidean')
# cosine (0.000025), (min_samples=2, eps=5.5, metric='euclidean'), (min_samples=3, eps=0.01, metric='cosine')

# clf.fit(embeddings_all['embeddings'])
clf.fit(reduced_embeddings)
preds = clf.labels_
print(preds)
print('Number of clusters:', len(set(preds)))

for i in range(0, len(preds)):
    cluster_dir = 'cluster ' + str(preds[i]) + '/'

    if not os.path.exists(dir_output_audio_step5 + cluster_dir):
        os.makedirs(dir_output_audio_step5 + cluster_dir)
    
    shutil.copyfile(dir_output_audio_step3 + embeddings_all['filename'][i], dir_output_audio_step5 + cluster_dir + embeddings_all['filename'][i])

# Calculate silhouette score to evaluate the quality of clustering
silhouette_avg = silhouette_score(embeddings_all['embeddings'], preds)
print("The average silhouette_score is :", silhouette_avg)
    
torch.cuda.empty_cache()
gc.collect()
# device = cuda.get_current_device()
# device.reset()
print('Finish.')

In [ ]:
from kneed import KneeLocator

# %%timeit
### Cluster Voice Embeddings ###
# Delete all clusters folders
for filename in sorted(os.listdir(dir_output_audio_step5)):
    if 'cluster' in filename:
        shutil.rmtree(dir_output_audio_step5+filename)

wcss = []
silhouette_scores = []
clusters_list_i = []
for num_clusters in range(6, 13):  # testing number of clusters from 1 to 10
    clusters_list_i.append(num_clusters)
    # KMeans Clustering
    kmeans = KMeans(n_clusters=num_clusters, init='k-means++', n_init='auto', max_iter=2000, random_state=42)
    kmeans.fit(embeddings_all['embeddings'])
    labels_clusters = kmeans.fit_predict(embeddings_all['embeddings'])
    wcss.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(embeddings_all['embeddings'], labels_clusters))

# Assuming 'wcss' is a list of your Within-Cluster-Sum-of-Squares values and 'range_n_clusters' is the list of number of clusters
knee_locator = KneeLocator(clusters_list_i, wcss, curve='convex', direction='decreasing')
optimal_num_clusters = knee_locator.elbow
print('Optimal Clusters:', optimal_num_clusters)

# Train final Kmeans using the optimal number of clusters
clf = KMeans(n_clusters=optimal_num_clusters, init='k-means++', n_init='auto', max_iter=2000, random_state=42)
clf.fit(embeddings_all['embeddings'])
# labels_clusters = clf.fit_predict(embeddings_all['embeddings'])
preds = clf.labels_

# clf = KMeans(n_clusters=n_clusters, n_init=100, max_iter=600, random_state=0)
# clf.fit(embeddings_all['embeddings'])
# preds = clf.labels_
# print(preds)
# print('Number of clusters:', len(set(preds)))

# Copy to correct cluster
for i in range(0, len(preds)):
    cluster_dir = 'cluster ' + str(preds[i]) + '/'
    if not os.path.exists(dir_output_audio_step5 + cluster_dir):
        os.makedirs(dir_output_audio_step5 + cluster_dir)
    shutil.copyfile(dir_output_audio_step3 + embeddings_all['filename'][i], dir_output_audio_step5 + cluster_dir + embeddings_all['filename'][i])

print('Finish.')